In [61]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [62]:
# 크롤링 문서 요청해서 응답객체 반환
url="https://www.mcst.go.kr/kor/s_culture/festival/festivalList.jsp?pSeq=&pRo=&pOrder=01up&pPeriod=&fromDt=20180101&toDt=20181231&pSido=15&pSearchType=02&pSearchWord=%EB%AA%A9%ED%8F%AC&pCurrentPage=1"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")

In [63]:
## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
# ul class:prdList grid4
ul=bs_obj.find("ul",{"class":"mediaWrap color01"})
## 품목 1개를 담고 있는 div 태그 추출
## div class:box
prd_boxes= ul.findAll("div", {"class":"text"})


In [64]:
# box 안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로 반환하는 함수
def get_product_info(text) :
    p_tag = text.find("p",{"class":"title"})
    price_ul = text.find("ul")
    price_span = price_ul.findAll("li")
    
    # 가격 추출 코드

    # 최종 data 추출 후 출력
    
    return{"축제이름":p_tag.text,"기간":price_span[0].text,"장소":price_span[1].text}


In [65]:
for text in prd_boxes :
    product_info = get_product_info(text)
    print(product_info)

{'축제이름': '목포 문화재야행 2018', '기간': '기간 : 2018.10.26~27 / 18:00~23:00', '장소': '장소 : 전라남도 목포시'}
{'축제이름': '목포 문화재야행 2018', '기간': '기간 : 1차 : 2018.09.07~08, 2차 : 2018.10.26~27 / 18:00~23:00', '장소': '장소 : 전라남도 목포시'}
{'축제이름': '목포항구축제 2018', '기간': '기간 : 2018. 10.5(금) ~10.7(일) / 10:00~22:00', '장소': '장소 : 전라남도 목포시'}
{'축제이름': '목포세계마당페스티벌 2018', '기간': '기간 : 2018.08.31 ~2018.09.02 / 18:00~23:00', '장소': '장소 : 전라남도 목포시'}
{'축제이름': '목포 이순신 수군 문화제 2018', '기간': '기간 : 2018.04.07~2018.04.08', '장소': '장소 : 전라남도 목포시'}


In [66]:
# 전달된 url 페이지에 접근해서 해당 페이지의 전체 상품 데이터를 추출 한 후 각 상품마다  get_product_info()함수를 호출해서
# 각 상품에 대한 추출 정보를 받아옴
def get_page_products(url) :
    url= url
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")
    
    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"mediaWrap color01"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes= ul.findAll("div", {"class":"text"}) # 1개 페이지의 전체 상품
    prod_info_list = [get_product_info(text) for text in prd_boxes]
    # 저장되어 있는 리스트
    return prod_info_list

In [67]:
# 여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "https://www.mcst.go.kr/kor/s_culture/festival/festivalList.jsp?pSeq=&pRo=&pOrder=01up&pPeriod=&fromDt=20180101&toDt=20181231&pSido=15&pSearchType=02&pSearchWord=%EB%AA%A9%ED%8F%AC&pCurrentPage=" # 페이지 번호를 제외한 공통 url 문자열
#print(get_page_products(url))

df_fin = pd.DataFrame()
for i in range(1,4) :
    # url 완성 : page 번호를 추가 i 변수 값을 활용
    urlfin = url + str(i)
    page_product = get_page_products(urlfin)
    df = pd.DataFrame(page_product) # 현재 url로 접속한 페이지의 대해 수집된 제품정보를 df로 생성(한 개 페이지 제품)
    df_fin = pd.concat([df_fin,df],axis=0,ignore_index=True)

df_fin

,축제이름,기간,장소
0,목포 문화재야행 2018,기간 : 2018.10.26~27 / 18:00~23:00,장소 : 전라남도 목포시
1,목포 문화재야행 2018,"기간 : 1차 : 2018.09.07~08, 2차 : 2018.10.26~27 / ...",장소 : 전라남도 목포시
2,목포항구축제 2018,기간 : 2018. 10.5(금) ~10.7(일) / 10:00~22:00,장소 : 전라남도 목포시
3,목포세계마당페스티벌 2018,기간 : 2018.08.31 ~2018.09.02 / 18:00~23:00,장소 : 전라남도 목포시
4,목포 이순신 수군 문화제 2018,기간 : 2018.04.07~2018.04.08,장소 : 전라남도 목포시


In [68]:
df_fin.to_csv('./crawl_data/목포_2018_축제.csv')